# [실습] Agent의 Planning ReAct 알아보기

`ToolNode`와 `ToolCondition`을 통해 툴을 실행하는 에이전트 구조를 만들어 보았습니다.

이 구조는 랭그래프에서 ReAct 에이전트라는 형태로 구현이 되어 있습니다.

In [ ]:
!pip install langgraph langchain langchain_google_genai langchain_community langchain_experimental

API 키와 LLM을 설정합니다.

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

LLM에서 사용할 툴을 설정합니다.

In [ ]:
# Tavily API
os.environ['TAVILY_API_KEY'] = 'tvly-xxxx'


from langchain_tavily import TavilySearch
from langchain_core.tools import tool
import random

tavily_search = TavilySearch(
    max_results=5)

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")


@tool
def counsel(problem:str) -> str: # 나중에는 LLM 기반의 모듈이나, 개별 에이전트로 처리 가능
    "고민에 대한 답을 예/아니오로 얻습니다."
    if random.random()>=0.5:
        return '네!'
    else:
        return '아니오.'

print(current_date.invoke({}))
print(counsel.invoke({'problem':'아무 고민'}))




이번에는 새로운 툴을 추가해 보겠습니다.   
`Python_REPL`은 파이썬 쉘을 이용해서 코드를 실행하는 기능으로, ChatGPT의 코드 인터프리터와 유사합니다.

- LangChain 0.3까지는 지원되었으나, 해당 기능은 최근 랭체인에서 지원이 종료되어 직접 구현합니다.

In [ ]:
from langchain_core.tools import tool
from io import StringIO
import sys
from contextlib import redirect_stdout

def python_repl(code: str) -> str:
    """
    Python 코드를 실행합니다.
    결과를 보려면 반드시 print()를 사용하세요.
    
    Args:
        code: 실행할 Python 코드
    
    Returns:
        실행 결과
    """
    # 코드 정리
    code = code.strip().strip('`')
    if code.startswith('python'):
        code = code[6:].strip()
    
    # stdout 캡처 (context manager 사용으로 안전하게)
    buffer = StringIO()
    
    try:
        # redirect_stdout을 사용하면 자동으로 복원됨
        with redirect_stdout(buffer):
            exec(code, globals())
        
        # 결과 가져오기
        output = buffer.getvalue()
        return output if output else "실행 완료 (출력 없음)"
        
    except Exception as e:
        return f"Error: {type(e).__name__}: {str(e)}"

@tool
def repl_tool(code: str) -> str:
    """
    파일 시스템 처리, 외부 API 호출 등의 다양한 작업을 위해
    Python 코드를 실행합니다.
    결과를 보려면 반드시 print()를 사용하세요.
    """
    result = python_repl(code)
    return result

# 테스트
print("=== 테스트 시작 ===")
print(python_repl("print(2 + 2)"))
print("=== 이 메시지가 보여야 합니다 ===")

React Agent는 `langgraph.prebuilt` 에서 바로 실행할 수 있습니다.

In [ ]:
from langgraph.prebuilt import create_react_agent

tools = [tavily_search, current_date, counsel, repl_tool]

graph = create_react_agent(llm, tools=tools)


# LangChain V1에서는 아래 코드로 변경
# from langchain.agents import create_agent
# agent = create_agent(llm, tools=tools, system_prompt = prompt)

In [ ]:
graph

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

graph.invoke({'messages':[HumanMessage(content="안녕하세요?")]})

In [ ]:
for data in graph.stream(
    {'messages':[
        HumanMessage(content='''
오늘 날짜 확인해서 txt 파일로 저장해, 파일 이름은 날짜.txt로 해.''')]},
    stream_mode='updates'):
    print(data)

이번에는 Tool Node, ToolCondition 등의 Prebuilt 없이 구현해 보겠습니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages : Annotated[list, add_messages]   # 메시지 맥락을 저장하는 리스트


In [ ]:
llm_with_tools = llm.bind_tools(tools)
llm_with_tools

In [ ]:
from langchain_core.messages import ToolMessage

tool_list = {tool.name: tool for tool in tools}
# tool 목록 dict로 생성

def tool_node(state):
    tool_outputs = []
    tool_call_msgs = state['messages'][-1]
    # 마지막 메시지: 툴 콜링 메시지

    for tool_call in tool_call_msgs.tool_calls:
    # 여러 개의 툴 콜이 필요한 경우를 고려해 for로 표시
        tool_exec = tool_list[tool_call['name']]
        # 실제 함수 찾기
        tool_result = tool_exec.invoke(tool_call)
        # tool 실행 결과 얻기 (결과는 ToolMessage 타입: 1번 실습 참고)
        tool_outputs.append(tool_result)

    return {'messages': tool_outputs}

def agent(state):

    # system_prompt = SystemMessage("주어진 툴을 사용하여, 사용자의 질문에 답하세요.")

    # ReAct 목적에 충실한 버전
    system_prompt = SystemMessage("""주어진 툴을 사용하여, 사용자의 질문에 답하세요.
툴을 실행하기 전, 직전까지의 결과의 의미를 파악하고 맥락에 맞게 다음 툴을 실행하기 위한 Planning Step을 메시지에 포함해 출력하세요.
파일 시스템 접근은 파이썬 코드를 실행하여 처리하세요.
에러가 발생하면, 전략을 바꿔 다시 실행하세요.""")


    response = llm_with_tools.invoke([system_prompt] + state["messages"])
    # Tool Calling이 필요한 경우와 필요하지 않은 경우를 구분할할 필요
    return {'messages': response}

def tool_needed(state):

    last_msg = state['messages'][-1]
    if last_msg.tool_calls: # 툴 콜링이 필요하면
        return "continue"
    else:
        return "finish"

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("tools", tool_node)

builder.add_edge(START, 'agent'),
builder.add_conditional_edges("agent",
                              tool_needed,
                               {"continue": "tools","finish": END})
builder.add_edge("tools", "agent")

In [ ]:
graph = builder.compile()
graph

In [ ]:
response = graph.invoke({'messages':[HumanMessage(content="패스트캠퍼스 랭그래프 과정 검색해서 소개해줘.")]})
response

In [ ]:
for data in graph.stream(
    {'messages':[HumanMessage(content='''
    오늘 날짜 확인해서 txt 파일로 저장해, 파일 이름은 날짜.txt로 해.''')]}, stream_mode='updates'):
    print(data)
    print('----------')
